In [84]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import os

In [85]:
# Agent for News Analysis
# Returns:
# 1. The significance level of the news (High, Medium, Low)
# 2. The main topic of the article
def news_analyst_agent(article):
    # TODO : Change the prompt
    prompt = "Fill here the prompt you want to use to analyze the news"
    
    api_key = os.environ["MISTRAL_API_KEY"]
    model = "mistral-large-latest"

    client = MistralClient(api_key=api_key)

    messages = [
        ChatMessage(role="system", content=prompt),
        ChatMessage(role="user", content="Content of the article : " + article)
    ]
    
    chat_response = client.chat(
        model=model,
        messages=messages,
    )

    print(chat_response.choices[0].message.content)

In [86]:
def news_agent(article):
    significance, topic = news_analyst_agent(article)
    
    dispatch_agent(topic)

In [87]:
import json

# Agent that tells the user responsible to the text given
def dispatch_agent(text):
    results = internal_retriever_agent(text)
    
    return None

In [88]:
from OpenRAG.src.openrag.chunk_vectorization.chunk_vectorization import get_vectorizer
from OpenRAG.src.openrag.vectordb.milvus_adapter import init_milvus_connection
from pymilvus import Collection

# Agent to retrieve internal information
# Returns:
# 1. Several contexts linked to the provided text
def internal_retriever_agent(text):
    vectorizer = get_vectorizer('mistral')
    query_vector = vectorizer.vectorize(text)
    
    init_milvus_connection()
    
    collection_name = "mistral_collection"
    collection = Collection(name=collection_name)
    
    n_neighbors = 10
    results = collection.search([query_vector], "vector", param={"metric_type": "L2", "params":{}}, limit=n_neighbors, expr="source == 'Internal'")
    
    final_indices = []
    for result in results[0]:
        if result.id not in final_indices and len(final_indices) + 1 <= n_neighbors:
            final_indices.append([result.id, result.distance])
            prev_index = result.id - 1
            if prev_index >= 0 and prev_index not in final_indices and len(final_indices) + 1 <= n_neighbors:
                final_indices.append([prev_index, result.distance])
            next_index = result.id + 1
            if next_index not in final_indices and len(final_indices) + 1 <= n_neighbors:
                final_indices.append([next_index, result.distance])

    results = final_indices
    
    answer_chunks = []
    for hit in results:
        print(find_chunks(hit[0]))
        answer_chunk = find_chunks(hit[0])
        answer_chunks.append(answer_chunk['text'])
    
    return answer_chunks

In [89]:
def find_chunks(id, path = "Data/Internal/Company StIT/"):
    """
    Find the chunk based on the given id.
    """
    global_indexing = json.load(open("global_indexing.json", "r"))
    for key, value in global_indexing.items():
        start_idx = value["start"]
        end_idx = value["end"]
        if start_idx <= id <= end_idx:
            index_in_file = id - start_idx
            data_dict_file = json.load(open(path + key + "_chunks.json", "r"))
            data_dict_file["chunk_" + str(index_in_file)]["document"] = key + ".docx"
            return data_dict_file["chunk_" + str(index_in_file)]
    return None

In [90]:
dispatch_agent("Something about the actual leader of the company")

{'text': "technology industry, trends, and market dynamics. In-depth knowledge of the technology industry, trends, and market dynamics. 5. Experience in fundraising, mergers and acquisitions, and corporate governance. 6. Ability to inspire, motivate, and develop teams and individuals.  Education: Master of Business Administration (MBA) INSEAD, Fontainebleau, France Graduated: June 2008  Master of Science (MSc) in Computer Science École Normale Supérieure, Paris, France Graduated: June 2003  Work Experience: CEO StIT, Paris, France March 2015 - Present  1. Led the company's strategic direction, vision, and growth, resulting in a 5x increase in revenue and a successful Series C funding round. 2. Built and managed a high-performing executive team, fostering collaboration, accountability, and results. Built and managed a high-performing executive team, fostering collaboration, accountability, and results.", 'page': 34, 'sentence_num': 2, 'sentences_page': 2, 'document': 'CV THOMAS LEFÈVRE.

In [91]:
# Function scanning the response from the LLM for a specific action call
def scan_response(response):
    # TODO : Implement the different cases for the different actions and run the corresponding agent
    pass